In this document, we'll go over some probability and statistics, and then 
use that to model the data durability in a system storing data using erasure
coding.

# Probability and Statistics

## Failure Rate

The distinction between failure rates and the probability of failure causes
lots of confusion.  I have to think about it carefully each time I come back
to the topic.

The *failure rate* for a widget over a period of time is the average number 
of failures in that period per widget.  An annual failure 
rate of 0.25 means that on average, there are 0.25 failures per widget.
If you have 100 widgets for one year, there would be an average of 25 failures per year.

A failure rate of 0.25 is frequently written as 25%.

It's counter-intuitive, but failure rates for unreliable widgets can be over 
1.0 (100%).  An annual failure rate of 12.0 (1200%) would mean that on average 
you would see 12 failures per year per widget.  An annual failure rate of 12.0 (1200%)
is the same thing as a monthly failure rate of 1.0 (100%), and is the same thing as a daily
failure rate of 0.0333 (3.33%).

## Probability of Failure

Assuming that the probability of separate failures is independent, the probability of failure 
over a period of time is modeled with the
[Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution).

If you have an annual failure rate of 2.0 (200%), there may be 0, 1, 2, 3, or more
failures in any year.  The probability of exactly $k$ failures in one year is given
by the formula:

$$\text{probability of exactly k failures} = e^{-\lambda} \: \frac{\lambda^k}{k!}$$

This formula tells us that the probability of no failures is $0.1353$, the probability of exactly one 
failure is $0.2707$, and so on:

Number of Failures | Probability
--- | ---
0 | 0.1353
1 | 0.2707
2 | 0.2707
3 | 0.1804
4 | 0.0902
5 | 0.0361
6 | 0.0120
... | ...

If you add up the infinite sequence of probabilites, it will add up to 1.0.

To calculate the probability of zero failures, you can simplify the formula:

$$\text{probability of 0 failures} = e^{-\lambda} \: \frac{\lambda^0}{0!} \: = \: \: e^{-\lambda}$$

The probability of having at least one failure is the sum of entries from 1 on, which is 
the same as one minus the probability of 0 failures:

$$\text{probability of one or more failures} \: = \: 1 \: - \: e^{-\lambda}$$

We'll use this formula later to calculate the probability of failures given a failure rate.

## Probability of $n$ Failures

So what if you have $n$ widgets, and you want to know if $k$ or more of them will fail?

Let's look at an example: You have three widgets with an annual failure rate of 0.25 (25%).  
What is the probability that 2 or more of the three widgets will fail in the year?

We'll use $P$ for the probability of a given widget failing at least once in the period.
Using the formula from the prevous section, that probability is $0.2212$:

$$P = 1 - e^{-0.25} \approx 0.2212$$

The probability of that widget not failing is $0.7788$:

$$\text{probability of not failing} = 1 - P \approx 0.7788$$

There are eight possible combinations of failure for the three widgets.  The first one is that none of them fail.  The probability for that is the product of the probabilities for each of the widgets not failing: $0.7788 \times 0.7788 \times 0.7788 = 0.4724$

We can compute the probability of all eight cases by taking the probability that each
widget will be OK or FAIL and multiplying them together.  The sum of the resulting probabilities
in the right column add up to 1.0:

A | A prob | B | B prob | C | C prob | Probability
--- | --- | --- | --- | --- | --- | ---
ok | 0.7788 | ok | 0.7788 | ok | 0.7788 | 0.4724
ok | 0.7788 | ok | 0.7788 | FAIL | 0.2212 | 0.1342
ok | 0.7788 | FAIL | 0.2212 | ok | 0.7788 | 0.1342
ok | 0.7788 | FAIL | 0.2212 | FAIL | 0.2212 | 0.0381
FAIL | 0.2212 | ok | 0.7788 | ok | 0.7788 | 0.1342
FAIL | 0.2212 | ok | 0.7788 | FAIL | 0.2212 | 0.0381
FAIL | 0.2212 | FAIL | 0.2212 | ok | 0.7788 | 0.0381
FAIL | 0.2212 | FAIL | 0.2212 | FAIL | 0.2212 | 0.0108

To get the probability of two or more failing, we add up the probabilities of all
rows that have two or more failures.  The rows with exactly two failures add up
to $0.1143$.  The one row with three failures has a probability of $0.0108$.  Those
add up to a probability of $0.1251$ for two or more failures.

You'll notice that all of the rows with two failures have the same probability,
which makes sense.  The number of those rows is given by: $\binom{3}{2}$, which 
is three.  (This is called the [Binomial Coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient).)

So the probability of getting exactly two failures is:

$$\binom{3}{2} \times P^2 \times (1 - P)^1$$

In general, the probability of getting exactly $k$ failures in $n$ widgets is:

$$\binom{n}{k} \times P^k \times (1 - P)^{(n - k)}$$

If you want more information on this, you can read about the [Probability Mass Function](https://en.wikipedia.org/wiki/Binomial_distribution#Probability_mass_function) for a binomial.

We can use this formula to summarize the table above by number of failures:

Number of Failures | Probability
--- | ---
0 | 0.4724
1 | 0.4025
2 | 0.1143
3 | 0.0108

# Data Durability

Now we get into calculating the durability of data stored with erasure
coding, assuming a failure rate for each shard, and independent failures
for each shard.

First, some naming.  We will use these names in the calculations:

* $S$ is the total number of shards (data plus parity)
* $R$ is the repair time for a shard in days: how long it takes to replace a shard after it fails
* $A$ is the annual failure rate of one shard
* $F$ is the failure rate of a shard in $R$ days
* $P$ is the probability of a shard failing at least once in $R$ days
* $D$ is the durability of data over $R$ days: not too many shards are lost

One of the assumptions we make is that it takes $R$ days to repair a failed
shard.  Let's start with a simpler problem and look at the data durability
over a period of $R$ days.  For a data loss to happen in this time period,
$P+1$ shards (or more) would have to fail.

We will use $A$ to denote the annual failure rate of individual shards.
Over one year, the chances that a shard will fail is evenly distributed over
all of the $R$-day periods in the year.  We will use $F$ to denote the failure
rate of one shard in an $R$-day period:

$$F = A\frac{R}{365}$$

The probability of failure of a single shard in R days is approximately $F$, when $F$ is small.
The exact value, from the Poisson distribution is:

$$P = 1 \: - \: e^{-F}$$

Given the probability of one shard failing, we can use the binomial distribution's 
probability mass function to calculate the probability of exactly $n$ of the $S$
shards failing:

$$\binom{S}{n} \: P^n \: (1-P)^{S-n}$$
        
We also lose data if more than n shards fail in the period.  To include those,
we can sum the above formula for n through S shards, to get the probability of
data loss in $R$ days:

$$\sum_{k=n}^{S} \binom{S}{k} \: P^k \: (1-P)^{S-k}$$
    
The durability in each period is inverse of that:

$$D = 1 \: - \: \sum_{k=n}^{S} \binom{S}{k} \: P^k \: (1-P)^{S-k}$$

Durability over the full year 
happens when there's durability in all of the periods, which is the product of
probabilities:

$$D ^ {365/R}$$

And that's the answer!
